In [ ]:
from django.db import models
from django.contrib.auth import get_user_model
from django.utils import timezone
from django.core.validators import MinValueValidator, MaxValueValidator
import uuid
import string
import random

User = get_user_model()


class SubscriptionPlan(models.Model):
    """Subscription plan model with tiered access levels"""
    PLAN_TYPES = (
        ('FREE', 'Personal - Free'),
        ('PRO', 'Personal - Pro'),
        ('MAX', 'Personal - Max'),
        ('TEAM', 'Business - Team'),
        ('ENTERPRISE', 'Business - Enterprise'),
    )

    plan_type = models.CharField(max_length=20, choices=PLAN_TYPES, unique=True)
    name = models.CharField(max_length=100)
    description = models.TextField()
    price = models.DecimalField(max_digits=10, decimal_places=2)
    ai_chat_limit = models.PositiveIntegerField(default=0)
    can_access_tutorials = models.BooleanField(default=False)
    can_access_library = models.BooleanField(default=False)
    can_access_news = models.BooleanField(default=False)
    can_access_premium = models.BooleanField(default=False)
    
    created_at = models.DateTimeField(auto_now_add=True)
    updated_at = models.DateTimeField(auto_now=True)
    
    def __str__(self):
        return self.name


class UserProfile(models.Model):
    """Extended user profile with credits and subscription details"""
    user = models.OneToOneField(User, on_delete=models.CASCADE, related_name='profile')
    subscription = models.ForeignKey(SubscriptionPlan, on_delete=models.SET_NULL, null=True, blank=True)
    credits = models.PositiveIntegerField(default=0)
    referral_code = models.CharField(max_length=16, unique=True, blank=True)
    referred_by = models.ForeignKey('self', on_delete=models.SET_NULL, null=True, blank=True, related_name='referrals')
    streak_days = models.PositiveIntegerField(default=0)
    last_login_date = models.DateField(null=True, blank=True)
    subscription_active = models.BooleanField(default=False)
    subscription_expiry = models.DateTimeField(null=True, blank=True)
    
    created_at = models.DateTimeField(auto_now_add=True)
    updated_at = models.DateTimeField(auto_now=True)

    def save(self, *args, **kwargs):
        # Generate unique referral code if not present
        if not self.referral_code:
            self.referral_code = self.generate_referral_code()
        
        # Check if it's a new login day and update streak
        self._update_streak()
        
        super().save(*args, **kwargs)
    
    def generate_referral_code(self):
        """Generate a random 16-character referral code"""
        chars = string.ascii_uppercase + string.digits
        code = ''.join(random.choice(chars) for _ in range(16))
        
        # Check if code already exists
        while UserProfile.objects.filter(referral_code=code).exists():
            code = ''.join(random.choice(chars) for _ in range(16))
        
        return code
    
    def _update_streak(self):
        """Update login streak based on consecutive days"""
        today = timezone.now().date()
        
        if not self.last_login_date:
            self.streak_days = 1
        elif (today - self.last_login_date).days == 1:
            # Consecutive day login
            self.streak_days += 1
        elif (today - self.last_login_date).days > 1:
            # Streak broken
            self.streak_days = 1
        # If same day login, no change to streak
        
        self.last_login_date = today
    
    def add_credits(self, amount):
        """Add credits to user account"""
        self.credits += amount
        self.save(update_fields=['credits', 'updated_at'])
    
    def use_credits(self, amount):
        """Use credits if available"""
        if self.credits >= amount:
            self.credits -= amount
            self.save(update_fields=['credits', 'updated_at'])
            return True
        return False
    
    def has_access_to(self, content_type):
        """Check if user has access to content type based on subscription"""
        if not self.subscription:
            return False
            
        if content_type == 'tutorials':
            return self.subscription.can_access_tutorials
        elif content_type == 'library':
            return self.subscription.can_access_library
        elif content_type == 'news':
            return self.subscription.can_access_news
        elif content_type == 'premium':
            return self.subscription.can_access_premium
        elif content_type == 'ai':
            # For AI, we check remaining chats in CreditTransaction
            return True
        return False
    
    def get_remaining_ai_chats(self):
        """Get remaining AI chats based on subscription and usage"""
        if not self.subscription:
            return 0
            
        # Get today's chat usage
        today = timezone.now().date()
        today_usage = CreditTransaction.objects.filter(
            user_profile=self,
            transaction_type='AI_CHAT',
            created_at__date=today
        ).count()
        
        return max(0, self.subscription.ai_chat_limit - today_usage)
    
    def __str__(self):
        return f"{self.user.email}'s Profile"


class CreditTransaction(models.Model):
    """Track all credit transactions"""
    TRANSACTION_TYPES = (
        ('SIGNUP', 'New User Signup'),
        ('REFERRAL', 'Referral Bonus'),
        ('STREAK', 'Login Streak Bonus'),
        ('CONTENT_ACCESS', 'Premium Content Access'),
        ('AI_CHAT', 'AI Chat Usage'),
        ('MANUAL', 'Manual Adjustment'),
        ('SUBSCRIPTION', 'Subscription Purchase'),
    )
    
    user_profile = models.ForeignKey(UserProfile, on_delete=models.CASCADE, related_name='transactions')
    amount = models.IntegerField()  # Can be positive (earned) or negative (spent)
    transaction_type = models.CharField(max_length=20, choices=TRANSACTION_TYPES)
    description = models.CharField(max_length=255)
    content_id = models.CharField(max_length=100, blank=True, null=True)  # Optional reference to content
    created_at = models.DateTimeField(auto_now_add=True)
    
    class Meta:
        indexes = [
            models.Index(fields=['user_profile', 'created_at']),
            models.Index(fields=['transaction_type']),
            models.Index(fields=['created_at']),
        ]
    
    def __str__(self):
        return f"{self.transaction_type}: {self.amount} credits for {self.user_profile.user.email}"


class Subscription(models.Model):
    """Track subscription payments and history"""
    STATUS_CHOICES = (
        ('ACTIVE', 'Active'),
        ('EXPIRED', 'Expired'),
        ('CANCELLED', 'Cancelled'),
        ('FAILED', 'Payment Failed'),
    )
    
    user_profile = models.ForeignKey(UserProfile, on_delete=models.CASCADE, related_name='subscription_history')
    plan = models.ForeignKey(SubscriptionPlan, on_delete=models.PROTECT)
    status = models.CharField(max_length=20, choices=STATUS_CHOICES, default='ACTIVE')
    start_date = models.DateTimeField(default=timezone.now)
    end_date = models.DateTimeField()
    payment_id = models.CharField(max_length=100, blank=True, null=True)  # PayPal payment ID
    auto_renew = models.BooleanField(default=False)
    
    created_at = models.DateTimeField(auto_now_add=True)
    updated_at = models.DateTimeField(auto_now=True)
    
    class Meta:
        indexes = [
            models.Index(fields=['user_profile', 'status']),
            models.Index(fields=['end_date']),
        ]
    
    def __str__(self):
        return f"{self.user_profile.user.email}'s {self.plan.name} subscription"
    
    def is_active(self):
        return self.status == 'ACTIVE' and self.end_date > timezone.now()

In [ ]:
from rest_framework import serializers
from .models import UserProfile, CreditTransaction, Subscription, SubscriptionPlan
from django.contrib.auth import get_user_model
from django.utils import timezone

User = get_user_model()


class SubscriptionPlanSerializer(serializers.ModelSerializer):
    class Meta:
        model = SubscriptionPlan
        fields = [
            'id', 'plan_type', 'name', 'description', 'price',
            'ai_chat_limit', 'can_access_tutorials', 'can_access_library',
            'can_access_news', 'can_access_premium'
        ]


class SubscriptionSerializer(serializers.ModelSerializer):
    plan = SubscriptionPlanSerializer(read_only=True)
    days_remaining = serializers.SerializerMethodField()
    
    class Meta:
        model = Subscription
        fields = [
            'id', 'plan', 'status', 'start_date', 'end_date',
            'auto_renew', 'days_remaining'
        ]
    
    def get_days_remaining(self, obj):
        if obj.end_date and obj.status == 'ACTIVE':
            now = timezone.now()
            if obj.end_date > now:
                return (obj.end_date - now).days
        return 0


class CreditTransactionSerializer(serializers.ModelSerializer):
    transaction_date = serializers.SerializerMethodField()
    
    class Meta:
        model = CreditTransaction
        fields = [
            'id', 'amount', 'transaction_type', 'description',
            'content_id', 'created_at', 'transaction_date'
        ]
    
    def get_transaction_date(self, obj):
        return obj.created_at.strftime('%Y-%m-%d %H:%M:%S')


class UserSerializer(serializers.ModelSerializer):
    class Meta:
        model = User
        fields = ['id', 'email', 'first_name', 'last_name', 'date_joined']


class UserProfileSerializer(serializers.ModelSerializer):
    user = UserSerializer(read_only=True)
    subscription_details = serializers.SerializerMethodField()
    remaining_ai_chats = serializers.SerializerMethodField()
    
    class Meta:
        model = UserProfile
        fields = [
            'id', 'user', 'credits', 'referral_code',
            'streak_days', 'subscription_active', 'subscription_details',
            'remaining_ai_chats', 'created_at', 'updated_at'
        ]
    
    def get_subscription_details(self, obj):
        active_subscription = Subscription.objects.filter(
            user_profile=obj, 
            status='ACTIVE',
            end_date__gt=timezone.now()
        ).first()
        
        if active_subscription:
            return SubscriptionSerializer(active_subscription).data
        return None
    
    def get_remaining_ai_chats(self, obj):
        return obj.get_remaining_ai_chats()


class CreditHistorySerializer(serializers.ModelSerializer):
    transactions = serializers.SerializerMethodField()
    
    class Meta:
        model = UserProfile
        fields = ['id', 'credits', 'transactions']
    
    def get_transactions(self, obj):
        # Get the most recent 50 transactions
        transactions = obj.transactions.all().order_by('-created_at')[:50]
        return CreditTransactionSerializer(transactions, many=True).data


class ReferralSerializer(serializers.Serializer):
    referral_code = serializers.CharField(max_length=16)
    
    def validate_referral_code(self, value):
        # Check if referral code exists
        if not UserProfile.objects.filter(referral_code=value).exists():
            raise serializers.ValidationError("Invalid referral code")
        
        # Make sure user isn't using their own code
        user = self.context.get('request').user
        if UserProfile.objects.filter(user=user, referral_code=value).exists():
            raise serializers.ValidationError("Cannot use your own referral code")
        
        return value

In [ ]:
from rest_framework import viewsets, generics, status, permissions
from rest_framework.response import Response
from rest_framework.decorators import action
from rest_framework.exceptions import PermissionDenied, ValidationError
from rest_framework.views import APIView
from django.utils import timezone
from django.db import transaction
from django.conf import settings
from django.contrib.auth import get_user_model
from django.shortcuts import get_object_or_404

from drf_yasg.utils import swagger_auto_schema
from drf_yasg import openapi

from .models import UserProfile, CreditTransaction, Subscription, SubscriptionPlan
from .serializers import (
    UserProfileSerializer, CreditHistorySerializer, 
    SubscriptionPlanSerializer, SubscriptionSerializer,
    ReferralSerializer
)
from .permissions import IsOwnerOrAdmin
from .throttles import UserProfileRateThrottle

User = get_user_model()


class UserProfileView(generics.RetrieveAPIView):
    """
    Get the profile of the currently authenticated user
    """
    serializer_class = UserProfileSerializer
    permission_classes = [permissions.IsAuthenticated]
    throttle_classes = [UserProfileRateThrottle]
    
    def get_object(self):
        user_profile, created = UserProfile.objects.get_or_create(user=self.request.user)
        
        # If this is a new profile, give signup bonus credits
        if created:
            # Give new users 50 credits
            user_profile.add_credits(50)
            CreditTransaction.objects.create(
                user_profile=user_profile,
                amount=50,
                transaction_type='SIGNUP',
                description='Welcome bonus for new user'
            )
            
            # Assign free plan
            free_plan = SubscriptionPlan.objects.filter(plan_type='FREE').first()
            if free_plan:
                user_profile.subscription = free_plan
                user_profile.subscription_active = True
                user_profile.save()
                
                # Create subscription record
                Subscription.objects.create(
                    user_profile=user_profile,
                    plan=free_plan,
                    status='ACTIVE',
                    start_date=timezone.now(),
                    # Free plan never expires
                    end_date=timezone.now() + timezone.timedelta(days=36500)  
                )
        
        return user_profile


class CreditHistoryView(generics.RetrieveAPIView):
    """
    Get the credit history for the currently authenticated user
    """
    serializer_class = CreditHistorySerializer
    permission_classes = [permissions.IsAuthenticated]
    
    def get_object(self):
        return get_object_or_404(UserProfile, user=self.request.user)


class SubscriptionPlanViewSet(viewsets.ReadOnlyModelViewSet):
    """
    View for listing and retrieving subscription plans
    """
    queryset = SubscriptionPlan.objects.all()
    serializer_class = SubscriptionPlanSerializer
    permission_classes = [permissions.IsAuthenticated]


class ContentAccessView(APIView):
    """
    Check if user has access to content and consume credits if needed
    """
    permission_classes = [permissions.IsAuthenticated]
    
    @swagger_auto_schema(
        request_body=openapi.Schema(
            type=openapi.TYPE_OBJECT,
            required=['content_type', 'content_id'],
            properties={
                'content_type': openapi.Schema(type=openapi.TYPE_STRING, description="Type of content (tutorials, library, news, premium, ai)"),
                'content_id': openapi.Schema(type=openapi.TYPE_STRING, description="ID of the content"),
                'credit_cost': openapi.Schema(type=openapi.TYPE_INTEGER, description="Cost in credits if applicable")
            }
        ),
        responses={
            200: openapi.Response(
                description="Access granted",
                schema=openapi.Schema(
                    type=openapi.TYPE_OBJECT,
                    properties={
                        'access_granted': openapi.Schema(type=openapi.TYPE_BOOLEAN),
                        'remaining_credits': openapi.Schema(type=openapi.TYPE_INTEGER),
                        'message': openapi.Schema(type=openapi.TYPE_STRING)
                    }
                )
            ),
            403: "Access denied"
        }
    )
    def post(self, request):
        content_type = request.data.get('content_type')
        content_id = request.data.get('content_id')
        credit_cost = int(request.data.get('credit_cost', 0))
        
        if not content_type or not content_id:
            return Response(
                {"error": "Content type and content ID are required"},
                status=status.HTTP_400_BAD_REQUEST
            )
        
        # Get user profile
        user_profile, _ = UserProfile.objects.get_or_create(user=request.user)
        
        # Check subscription access
        has_subscription_access = user_profile.has_access_to(content_type)
        
        # For AI chats, check daily limit
        if content_type == 'ai' and has_subscription_access:
            remaining_chats = user_profile.get_remaining_ai_chats()
            if remaining_chats <= 0:
                has_subscription_access = False
        
        # If subscription provides access, grant it
        if has_subscription_access:
            if content_type == 'ai':
                # Record AI chat usage
                CreditTransaction.objects.create(
                    user_profile=user_profile,
                    amount=0,  # No credits used, count towards daily limit
                    transaction_type='AI_CHAT',
                    description=f"AI chat usage",
                    content_id=content_id
                )
            
            return Response({
                'access_granted': True,
                'remaining_credits': user_profile.credits,
                'message': f"Access granted via {user_profile.subscription.name} subscription"
            })
        
        # If no subscription access, try to use credits
        if credit_cost > 0:
            # Check if user has enough credits
            if user_profile.credits >= credit_cost:
                # Use credits
                with transaction.atomic():
                    success = user_profile.use_credits(credit_cost)
                    if success:
                        # Record transaction
                        CreditTransaction.objects.create(
                            user_profile=user_profile,
                            amount=-credit_cost,
                            transaction_type='CONTENT_ACCESS',
                            description=f"Access to {content_type} content",
                            content_id=content_id
                        )
                        
                        return Response({
                            'access_granted': True,
                            'remaining_credits': user_profile.credits,
                            'message': f"{credit_cost} credits used for access"
                        })
        
        # If we get here, access is denied
        return Response(
            {
                'access_granted': False,
                'remaining_credits': user_profile.credits,
                'message': "Insufficient credits or subscription for access"
            },
            status=status.HTTP_403_FORBIDDEN
        )


class ReferralView(APIView):
    """
    Apply a referral code to get bonus credits
    """
    permission_classes = [permissions.IsAuthenticated]
    
    @swagger_auto_schema(
        request_body=ReferralSerializer,
        responses={
            200: openapi.Response(
                description="Referral successfully applied",
                schema=openapi.Schema(
                    type=openapi.TYPE_OBJECT,
                    properties={
                        'success': openapi.Schema(type=openapi.TYPE_BOOLEAN),
                        'credits_added': openapi.Schema(type=openapi.TYPE_INTEGER),
                        'total_credits': openapi.Schema(type=openapi.TYPE_INTEGER)
                    }
                )
            ),
            400: "Invalid referral code"
        }
    )
    def post(self, request):
        serializer = ReferralSerializer(data=request.data, context={'request': request})
        if serializer.is_valid():
            referral_code = serializer.validated_data['referral_code']
            
            # Get user profile
            user_profile, _ = UserProfile.objects.get_or_create(user=request.user)
            
            # Check if user already used a referral
            if user_profile.referred_by is not None:
                return Response(
                    {"error": "You have already used a referral code"},
                    status=status.HTTP_400_BAD_REQUEST
                )
            
            # Get referrer profile
            referrer_profile = get_object_or_404(UserProfile, referral_code=referral_code)
            
            with transaction.atomic():
                # Update user profile with referrer
                user_profile.referred_by = referrer_profile
                user_profile.save()
                
                # Add credits to both users
                REFERRAL_BONUS = 100  # Both users get 100 credits
                
                # Add credits to referred user
                user_profile.add_credits(REFERRAL_BONUS)
                CreditTransaction.objects.create(
                    user_profile=user_profile,
                    amount=REFERRAL_BONUS,
                    transaction_type='REFERRAL',
                    description=f"Referral bonus from {referrer_profile.user.email}"
                )
                
                # Add credits to referrer
                referrer_profile.add_credits(REFERRAL_BONUS)
                CreditTransaction.objects.create(
                    user_profile=referrer_profile,
                    amount=REFERRAL_BONUS,
                    transaction_type='REFERRAL',
                    description=f"Referral bonus for referring {user_profile.user.email}"
                )
            
            return Response({
                'success': True,
                'credits_added': REFERRAL_BONUS,
                'total_credits': user_profile.credits
            })
            
        return Response(serializer.errors, status=status.HTTP_400_BAD_REQUEST)


class StreakBonusView(APIView):
    """
    Claim streak bonus credits - this endpoint should be called when user logs in
    """
    permission_classes = [permissions.IsAuthenticated]
    
    def post(self, request):
        user_profile, _ = UserProfile.objects.get_or_create(user=request.user)
        
        # Check if already claimed today
        today = timezone.now().date()
        already_claimed = CreditTransaction.objects.filter(
            user_profile=user_profile,
            transaction_type='STREAK',
            created_at__date=today
        ).exists()
        
        if already_claimed:
            return Response(
                {"error": "Streak bonus already claimed today"},
                status=status.HTTP_400_BAD_REQUEST
            )
        
        # Calculate streak bonus - increases with consecutive days
        streak_bonus = 0
        if user_profile.streak_days == 1:
            streak_bonus = 5  # First day
        elif user_profile.streak_days == 2:
            streak_bonus = 10  # Second day
        elif user_profile.streak_days == 3:
            streak_bonus = 15  # Third day
        elif user_profile.streak_days == 4:
            streak_bonus = 20  # Fourth day
        elif user_profile.streak_days == 5:
            streak_bonus = 25  # Fifth day
        elif user_profile.streak_days == 6:
            streak_bonus = 30  # Sixth day
        elif user_profile.streak_days >= 7:
            streak_bonus = 50  # Seven or more days
        
        # Add credits
        user_profile.add_credits(streak_bonus)
        
        # Record transaction
        CreditTransaction.objects.create(
            user_profile=user_profile,
            amount=streak_bonus,
            transaction_type='STREAK',
            description=f"Day {user_profile.streak_days} login streak bonus"
        )
        
        return Response({
            'success': True,
            'credits_added': streak_bonus,
            'streak_days': user_profile.streak_days,
            'total_credits': user_profile.credits
        })

In [ ]:
from rest_framework import permissions


class IsOwnerOrAdmin(permissions.BasePermission):
    """
    Custom permission to only allow owners of an object or admins to access it
    """
    
    def has_object_permission(self, request, view, obj):
        # Check if user is admin
        if request.user.is_staff:
            return True
            
        # Check if the object has a user field directly
        if hasattr(obj, 'user'):
            return obj.user == request.user
            
        # Check if the object has a user_profile field that links to user
        if hasattr(obj, 'user_profile'):
            return obj.user_profile.user == request.user
            
        return False


class SubscriptionAccessPermission(permissions.BasePermission):
    """
    Permission to check if a user has adequate subscription for accessing content
    """
    
    def has_permission(self, request, view):
        # All authenticated users have basic permission
        if not request.user.is_authenticated:
            return False
            
        # For specific object-level permission, we'll check in has_object_permission
        return True
        
    def has_object_permission(self, request, view, obj):
        # Get content type from view or request
        content_type = getattr(view, 'content_type', None)
        if not content_type:
            content_type = request.query_params.get('content_type')
            
        if not content_type:
            return False
            
        # Get user profile
        try:
            profile = request.user.profile
        except:
            return False
            
        # Check if subscription allows access
        return profile.has_access_to(content_type)

In [ ]:
from rest_framework.throttling import UserRateThrottle, AnonRateThrottle


class UserProfileRateThrottle(UserRateThrottle):
    """
    Throttle for user profile endpoints
    Limits authenticated users to 60 requests per minute
    """
    rate = '60/minute'
    scope = 'user_profile'


class ContentAccessRateThrottle(UserRateThrottle):
    """
    Throttle for content access endpoints
    Limits authenticated users to 10 requests per minute to prevent spam
    """
    rate = '10/minute'
    scope = 'content_access'


class StreakBonusRateThrottle(UserRateThrottle):
    """
    Throttle for streak bonus endpoints
    Limits authenticated users to 5 requests per day to prevent abuse
    """
    rate = '5/day'
    scope = 'streak_bonus'


class ReferralRateThrottle(UserRateThrottle):
    """
    Throttle for referral endpoints
    Limits authenticated users to 3 requests per hour to prevent abuse
    """
    rate = '3/hour'
    scope = 'referral'


class ApiAnonThrottle(AnonRateThrottle):
    """
    Throttle for anonymous users
    Very strict limits to prevent abuse of the API
    """
    rate = '10/hour'
    scope = 'anon'

In [ ]:
from django.urls import path, include
from rest_framework.routers import DefaultRouter
from . import views

router = DefaultRouter()
router.register(r'subscription-plans', views.SubscriptionPlanViewSet)

urlpatterns = [
    path('', include(router.urls)),
    
    # User profile endpoints
    path('profile/', views.UserProfileView.as_view(), name='user-profile'),
    path('credit-history/', views.CreditHistoryView.as_view(), name='credit-history'),
    
    # Content access endpoint
    path('check-access/', views.ContentAccessView.as_view(), name='check-access'),
    
    # Bonus system endpoints
    path('apply-referral/', views.ReferralView.as_view(), name='apply-referral'),
    path('claim-streak-bonus/', views.StreakBonusView.as_view(), name='claim-streak-bonus'),
]

In [ ]:
import re
from django.utils import timezone
from django.urls import resolve
from django.http import HttpResponseForbidden
import json
import logging
from django.core.cache import cache

logger = logging.getLogger(__name__)


class SecurityHeadersMiddleware:
    """
    Add security headers to all responses
    """
    def __init__(self, get_response):
        self.get_response = get_response

    def __call__(self, request):
        response = self.get_response(request)
        
        # Add security headers
        response['X-Content-Type-Options'] = 'nosniff'
        response['X-Frame-Options'] = 'DENY'
        response['X-XSS-Protection'] = '1; mode=block'
        response['Referrer-Policy'] = 'same-origin'
        response['Content-Security-Policy'] = "default-src 'self'; script-src 'self'; connect-src 'self'; img-src 'self' data:; style-src 'self';"
        response['Permissions-Policy'] = "geolocation=(), microphone=(), camera=(), payment=()"
        response['Strict-Transport-Security'] = 'max-age=31536000; includeSubDomains'
        
        return response


class InputValidationMiddleware:
    """
    Validate input data for JSON requests
    """
    def __init__(self, get_response):
        self.get_response = get_response
        
        # Endpoints that should validate input
        self.validate_endpoints = [
            '/api/check-access/',
            '/api/apply-referral/',
            '/api/claim-streak-bonus/',
        ]
        
        # Patterns to detect potential SQL injections or XSS
        self.sql_injection_patterns = [
            r"['\"].*;.*--",  # SQL comment with semicolon
            r"OR\s+1\s*=\s*1",  # OR 1=1
            r"DROP\s+TABLE",  # DROP TABLE
            r"UNION\s+SELECT",  # UNION SELECT
            r"SELECT\s+.*\s+FROM",  # Basic SELECT query
        ]
        
        self.xss_patterns = [
            r"<script.*?>",  # Script tags
            r"javascript:",  # JavaScript protocol
            r"onload=",  # Event handlers
            r"onerror=",
            r"onclick=",
        ]

    def __call__(self, request):
        # Only check POST requests with JSON content
        if request.method == 'POST' and request.content_type == 'application/json':
            # Check if the path should be validated
            if any(endpoint in request.path for endpoint in self.validate_endpoints):
                try:
                    # Get request body
                    body = request.body.decode('utf-8')
                    
                    # Check for SQL injection or XSS patterns
                    for pattern in self.sql_injection_patterns + self.xss_patterns:
                        if re.search(pattern, body, re.IGNORECASE):
                            logger.warning(f"Suspicious request detected: {pattern} in {request.path}")
                            return HttpResponseForbidden("Invalid input detected")
                    
                    # JSON validation
                    try:
                        data = json.loads(body)
                    except json.JSONDecodeError:
                        return HttpResponseForbidden("Invalid JSON in request body")
                    
                except Exception as e:
                    logger.error(f"Error in InputValidationMiddleware: {str(e)}")
        
        return self.get_response(request)


class IPBasedThrottlingMiddleware:
    """
    Rate limiting based on IP address for anonymous users
    """
    def __init__(self, get_response):
        self.get_response = get_response
        self.rate_limit = 100  # requests per hour
        self.time_window = 3600  # 1 hour in seconds

    def __call__(self, request):
        # Skip for authenticated users
        if request.user.is_authenticated:
            return self.get_response(request)
            
        # Get client IP
        ip_address = self.get_client_ip(request)
        if not ip_address:
            return self.get_response(request)
            
        # Create cache key
        cache_key = f"ip_rate_limit:{ip_address}"
        
        # Get current request count
        request_count = cache.get(cache_key, 0)
        
        # Check if over limit
        if request_count >= self.rate_limit:
            return HttpResponseForbidden("Rate limit exceeded. Please try again later.")
            
        # Increment counter
        cache.set(cache_key, request_count + 1, self.time_window)
        
        return self.get_response(request)
        
    def get_client_ip(self, request):
        x_forwarded_for = request.META.get('HTTP_X_FORWARDED_FOR')
        if x_forwarded_for:
            ip = x_forwarded_for.split(',')[0]
        else:
            ip = request.META.get('REMOTE_ADDR')
        return ip


class StreakUpdaterMiddleware:
    """
    Update user streak on authenticated requests
    """
    def __init__(self, get_response):
        self.get_response = get_response

    def __call__(self, request):
        # Process request before view is called
        response = self.get_response(request)
        
        # Update streak for authenticated users
        if request.user.is_authenticated:
            try:
                # Only update once per day
                today = timezone.now().date()
                last_update_key = f"streak_update_{request.user.id}_{today}"
                
                # Check if already updated today
                if not cache.get(last_update_key):
                    # Get or create profile
                    from .models import UserProfile
                    profile, created = UserProfile.objects.get_or_create(user=request.user)
                    
                    # Update streak will be handled in the save method
                    profile.save()
                    
                    # Mark as updated today
                    cache.set(last_update_key, True, 24*60*60)  # 24 hours
            except Exception as e:
                logger.error(f"Error updating streak: {str(e)}")
        
        return response

In [ ]:
import re
from django.core.exceptions import ValidationError
from django.core.validators import RegexValidator


def validate_alphanumeric(value):
    """
    Validate that a value contains only alphanumeric characters
    """
    if not re.match(r'^[a-zA-Z0-9]+$', value):
        raise ValidationError('Value must contain only alphanumeric characters.')


def validate_referral_code(value):
    """
    Validate that a referral code is in the correct format
    16 characters, uppercase letters and numbers only
    """
    if not re.match(r'^[A-Z0-9]{16}$', value):
        raise ValidationError('Referral code must be 16 uppercase letters and numbers.')


def validate_content_type(value):
    """
    Validate content type is one of the allowed types
    """
    allowed_types = ['tutorials', 'library', 'news', 'premium', 'ai']
    if value not in allowed_types:
        raise ValidationError(f'Content type must be one of: {", ".join(allowed_types)}')


def validate_credit_cost(value):
    """
    Validate credit cost is a positive integer
    """
    try:
        cost = int(value)
        if cost < 0:
            raise ValidationError('Credit cost must be non-negative.')
    except (TypeError, ValueError):
        raise ValidationError('Credit cost must be an integer.')


# Reusable validators for models
alphanumeric_validator = RegexValidator(
    r'^[a-zA-Z0-9]+$',
    'Only alphanumeric characters are allowed.'
)

referral_code_validator = RegexValidator(
    r'^[A-Z0-9]{16}$',
    'Referral code must be 16 uppercase letters and numbers.'
)

In [ ]:
# Security Settings - Add to your settings.py

# Middleware configuration for security
MIDDLEWARE += [
    'app_name.middleware.SecurityHeadersMiddleware',
    'app_name.middleware.InputValidationMiddleware',
    'app_name.middleware.IPBasedThrottlingMiddleware',
    'app_name.middleware.StreakUpdaterMiddleware',
]

# CORS settings - Allow your NextJS frontend only
CORS_ALLOWED_ORIGINS = [
    "https://your-production-domain.com",
    "http://localhost:3000",  # Development only
]

CORS_ALLOW_CREDENTIALS = True

# Session Security
SESSION_COOKIE_SECURE = True  # Only send cookies over HTTPS
SESSION_COOKIE_HTTPONLY = True  # Prevent JavaScript access to cookies
SESSION_COOKIE_SAMESITE = 'Lax'  # CSRF protection
CSRF_COOKIE_SECURE = True
CSRF_COOKIE_HTTPONLY = True
CSRF_COOKIE_SAMESITE = 'Lax'

# REST Framework settings
REST_FRAMEWORK = {
    'DEFAULT_AUTHENTICATION_CLASSES': [
        'rest_framework_simplejwt.authentication.JWTAuthentication',
        'rest_framework.authentication.SessionAuthentication',
    ],
    'DEFAULT_PERMISSION_CLASSES': [
        'rest_framework.permissions.IsAuthenticated',
    ],
    'DEFAULT_THROTTLE_CLASSES': [
        'rest_framework.throttling.AnonRateThrottle',
        'rest_framework.throttling.UserRateThrottle',
    ],
    'DEFAULT_THROTTLE_RATES': {
        'anon': '20/hour',
        'user': '1000/day',
        'user_profile': '60/minute',
        'content_access': '10/minute',
        'streak_bonus': '5/day',
        'referral': '3/hour',
    },
    'DEFAULT_SCHEMA_CLASS': 'rest_framework.schemas.coreapi.AutoSchema',
    'DEFAULT_PAGINATION_CLASS': 'rest_framework.pagination.PageNumberPagination',
    'PAGE_SIZE': 50,
    'DEFAULT_RENDERER_CLASSES': [
        'rest_framework.renderers.JSONRenderer',
    ],
    'EXCEPTION_HANDLER': 'app_name.utils.custom_exception_handler',
}

# JWT Settings
SIMPLE_JWT = {
    'ACCESS_TOKEN_LIFETIME': timedelta(minutes=60),
    'REFRESH_TOKEN_LIFETIME': timedelta(days=7),
    'ROTATE_REFRESH_TOKENS': True,
    'BLACKLIST_AFTER_ROTATION': True,
    'ALGORITHM': 'HS256',
    'SIGNING_KEY': os.environ.get('JWT_SECRET_KEY', SECRET_KEY),
    'VERIFYING_KEY': None,
    'AUTH_HEADER_TYPES': ('Bearer',),
    'USER_ID_FIELD': 'id',
    'USER_ID_CLAIM': 'user_id',
    'AUTH_TOKEN_CLASSES': ('rest_framework_simplejwt.tokens.AccessToken',),
    'TOKEN_TYPE_CLAIM': 'token_type',
    'JTI_CLAIM': 'jti',
    'SLIDING_TOKEN_REFRESH_EXP_CLAIM': 'refresh_exp',
    'SLIDING_TOKEN_LIFETIME': timedelta(minutes=60),
    'SLIDING_TOKEN_REFRESH_LIFETIME': timedelta(days=7),
}

# Password validation
AUTH_PASSWORD_VALIDATORS = [
    {
        'NAME': 'django.contrib.auth.password_validation.UserAttributeSimilarityValidator',
    },
    {
        'NAME': 'django.contrib.auth.password_validation.MinimumLengthValidator',
        'OPTIONS': {
            'min_length': 10,
        }
    },
    {
        'NAME': 'django.contrib.auth.password_validation.CommonPasswordValidator',
    },
    {
        'NAME': 'django.contrib.auth.password_validation.NumericPasswordValidator',
    },
]

# Rate limiting and security
SECURE_PROXY_SSL_HEADER = ('HTTP_X_FORWARDED_PROTO', 'https')
SECURE_SSL_REDIRECT = True  # Make sure to disable in development
SECURE_HSTS_SECONDS = 31536000  # 1 year
SECURE_HSTS_INCLUDE_SUBDOMAINS = True
SECURE_HSTS_PRELOAD = True
SECURE_CONTENT_TYPE_NOSNIFF = True
SECURE_BROWSER_XSS_FILTER = True
X_FRAME_OPTIONS = 'DENY'

# Database Connection Pooling for high write throughput
DATABASES = {
    'default': {
        'ENGINE': 'django.db.backends.postgresql',
        'NAME': os.environ.get('DB_NAME', 'dbname'),
        'USER': os.environ.get('DB_USER', 'dbuser'),
        'PASSWORD': os.environ.get('DB_PASSWORD', 'dbpassword'),
        'HOST': os.environ.get('DB_HOST', 'localhost'),
        'PORT': os.environ.get('DB_PORT', '5432'),
        'CONN_MAX_AGE': 600,  # 10 minutes
        'OPTIONS': {
            'connect_timeout': 10,
            'application_name': 'django_app',
        },
        'ATOMIC_REQUESTS': True,  # Transaction per HTTP request
    }
}

# Cache settings
CACHES = {
    'default': {
        'BACKEND': 'django_redis.cache.RedisCache',
        'LOCATION': os.environ.get('REDIS_URL', 'redis://127.0.0.1:6379/1'),
        'OPTIONS': {
            'CLIENT_CLASS': 'django_redis.client.DefaultClient',
            'SOCKET_CONNECT_TIMEOUT': 5,
            'SOCKET_TIMEOUT': 5,
            'CONNECTION_POOL_KWARGS': {'max_connections': 100},
            'PASSWORD': os.environ.get('REDIS_PASSWORD', None),
        }
    }
}

# Session cache
SESSION_ENGINE = 'django.contrib.sessions.backends.cache'
SESSION_CACHE_ALIAS = 'default'

In [ ]:
from django.db.models.signals import post_save, pre_save
from django.dispatch import receiver
from django.contrib.auth import get_user_model
from django.utils import timezone
from django.db import transaction
from .models import UserProfile, Subscription

User = get_user_model()


@receiver(post_save, sender=User)
def create_user_profile(sender, instance, created, **kwargs):
    """
    Create a UserProfile when a new User is created
    """
    if created:
        with transaction.atomic():
            # Generate referral code in the save method
            UserProfile.objects.create(user=instance)


@receiver(pre_save, sender=Subscription)
def update_user_profile_subscription(sender, instance, **kwargs):
    """
    Update the user's profile subscription details when subscription changes
    """
    if instance.pk:  # Existing subscription
        try:
            old_instance = Subscription.objects.get(pk=instance.pk)
            
            # Only update if status changed
            if old_instance.status != instance.status or old_instance.end_date != instance.end_date:
                # Update user profile subscription status
                user_profile = instance.user_profile
                
                if instance.status == 'ACTIVE' and instance.end_date > timezone.now():
                    user_profile.subscription = instance.plan
                    user_profile.subscription_active = True
                    user_profile.subscription_expiry = instance.end_date
                else:
                    # If subscription is no longer active, revert to free plan
                    from .models import SubscriptionPlan
                    free_plan = SubscriptionPlan.objects.filter(plan_type='FREE').first()
                    
                    user_profile.subscription = free_plan
                    user_profile.subscription_active = False
                    user_profile.subscription_expiry = None
                
                user_profile.save(update_fields=['subscription', 'subscription_active', 'subscription_expiry'])
        except Subscription.DoesNotExist:
            pass
    else:  # New subscription
        # New subscriptions are handled in the view
        pass

In [ ]:
from django.contrib import admin
from .models import UserProfile, CreditTransaction, Subscription, SubscriptionPlan


@admin.register(SubscriptionPlan)
class SubscriptionPlanAdmin(admin.ModelAdmin):
    list_display = ('name', 'plan_type', 'price', 'ai_chat_limit')
    list_filter = ('can_access_tutorials', 'can_access_library', 'can_access_news', 'can_access_premium')
    search_fields = ('name', 'description')


@admin.register(UserProfile)
class UserProfileAdmin(admin.ModelAdmin):
    list_display = ('user', 'credits', 'referral_code', 'streak_days', 'subscription_active')
    list_filter = ('subscription_active', 'subscription')
    search_fields = ('user__email', 'user__username', 'referral_code')
    raw_id_fields = ('user', 'referred_by', 'subscription')
    readonly_fields = ('referral_code',)
    
    def get_queryset(self, request):
        # Prefetch related objects to reduce database queries
        return super().get_queryset(request).select_related(
            'user', 'subscription', 'referred_by'
        )


@admin.register(CreditTransaction)
class CreditTransactionAdmin(admin.ModelAdmin):
    list_display = ('user_profile', 'amount', 'transaction_type', 'created_at')
    list_filter = ('transaction_type', 'created_at')
    search_fields = ('user_profile__user__email', 'description', 'content_id')
    raw_id_fields = ('user_profile',)
    date_hierarchy = 'created_at'
    
    def get_queryset(self, request):
        # Prefetch related objects to reduce database queries
        return super().get_queryset(request).select_related('user_profile__user')


@admin.register(Subscription)
class SubscriptionAdmin(admin.ModelAdmin):
    list_display = ('user_profile', 'plan', 'status', 'start_date', 'end_date', 'auto_renew')
    list_filter = ('status', 'plan', 'auto_renew', 'start_date', 'end_date')
    search_fields = ('user_profile__user__email', 'payment_id')
    raw_id_fields = ('user_profile', 'plan')
    date_hierarchy = 'start_date'
    
    def get_queryset(self, request):
        # Prefetch related objects to reduce database queries
        return super().get_queryset(request).select_related(
            'user_profile__user', 'plan'
        )

In [ ]:
import logging
from rest_framework.views import exception_handler
from rest_framework.response import Response
from rest_framework import status
from django.http import Http404
from django.core.exceptions import ValidationError as DjangoValidationError
from django.core.exceptions import PermissionDenied as DjangoPermissionDenied

logger = logging.getLogger(__name__)


def custom_exception_handler(exc, context):
    """
    Custom exception handler for DRF.
    Adds more detail to error responses and logs exceptions.
    """
    # Call REST framework's default exception handler first
    response = exception_handler(exc, context)
    
    # If DRF couldn't handle it, try to handle Django exceptions
    if response is None:
        if isinstance(exc, Http404):
            response = Response(
                {'error': 'Not found', 'detail': str(exc)},
                status=status.HTTP_404_NOT_FOUND
            )
        elif isinstance(exc, DjangoValidationError):
            response = Response(
                {'error': 'Validation Error', 'detail': exc.messages},
                status=status.HTTP_400_BAD_REQUEST
            )
        elif isinstance(exc, DjangoPermissionDenied):
            response = Response(
                {'error': 'Permission Denied', 'detail': str(exc)},
                status=status.HTTP_403_FORBIDDEN
            )
        else:
            # Unexpected error - log it
            view = context.get('view', None)
            view_name = view.__class__.__name__ if view else 'Unknown'
            logger.error(
                f'Unhandled exception in {view_name}: {exc}',
                exc_info=True,
                extra={
                    'view': view_name,
                    'request': context.get('request', None)
                }
            )
            
            # Return generic error message
            response = Response(
                {'error': 'Internal Server Error'},
                status=status.HTTP_500_INTERNAL_SERVER_ERROR
            )
    
    # Add additional information for specific error types
    if response is not None and response.status_code == 400:
        if 'detail' in response.data and isinstance(response.data['detail'], list):
            # Format validation errors in a consistent way
            response.data = {
                'error': 'Validation Error',
                'detail': response.data['detail']
            }
    
    return response


def generate_index_recommendations():
    """
    Function to generate index recommendations for high write throughput
    For documentation purposes
    """
    recommendations = [
        """
        RECOMMENDED INDEXES FOR HIGH WRITE THROUGHPUT:
        
        1. For CreditTransaction:
           - Compound index on (user_profile_id, created_at) for efficient user history queries
           - Index on (transaction_type) for filtering transactions by type
           - Index on (created_at) for time-based filtering and reporting
        
        2. For UserProfile:
           - Index on (referral_code) for quick referral lookup
           - Index on (user_id) for user profile retrieval
        
        3. For Subscription:
           - Compound index on (user_profile_id, status) for active subscription lookup
           - Index on (end_date) for subscription expiration checks
           
        CREATE INDEX credit_transaction_user_profile_created_at_idx 
            ON credit_transaction(user_profile_id, created_at DESC);
            
        CREATE INDEX credit_transaction_type_idx 
            ON credit_transaction(transaction_type);
            
        CREATE INDEX credit_transaction_created_at_idx 
            ON credit_transaction(created_at);
            
        CREATE INDEX user_profile_referral_code_idx 
            ON user_profile(referral_code);
            
        CREATE INDEX subscription_user_status_idx 
            ON subscription(user_profile_id, status);
            
        CREATE INDEX subscription_end_date_idx 
            ON subscription(end_date);
        """
    ]
    
    return recommendations

In [ ]:
from django.apps import AppConfig


class CreditsConfig(AppConfig):
    default_auto_field = 'django.db.models.BigAutoField'
    name = 'credits'  # Replace with your actual app name

    def ready(self):
        # Import signals
        import credits.signals  # Replace with your actual app name

Let me complete the URLs file and add middleware configurations for security:

Now, let's add middleware configuration and security settings:

Let's create a validator module for input validation:

Let's add security settings to include in your Django settings.py:

Now, let's create a signals file to handle model events:

Let's create an admin file to manage the models through Django admin:

Finally, let's add a custom exception handler for the API:

Let's add app configuration to set up the signals:

## System Summary and Implementation Guide

I've created a comprehensive Django REST Framework setup for your credits and subscription system. Here's an overview of what's been implemented:

### Models
1. **UserProfile**: Extends the standard Django user model with credit points, referral code, subscription details, and streak tracking.
2. **SubscriptionPlan**: Defines different subscription tiers (Free, Pro, Max, Team, Enterprise) with their features.
3. **CreditTransaction**: Tracks all credit transactions with detailed history and categorization.
4. **Subscription**: Manages subscription history, including payment tracking for PayPal integration.

### API Endpoints
1. `/api/profile/`: Get the current user's profile with credit balance and subscription info.
2. `/api/credit-history/`: Get detailed credit transaction history.
3. `/api/check-access/`: Check if user has access to specific content based on credits or subscription.
4. `/api/apply-referral/`: Apply a referral code to get bonus credits.
5. `/api/claim-streak-bonus/`: Claim login streak bonus credits.
6. `/api/subscription-plans/`: List available subscription plans.

### Security Features
1. **Input Validation**: Middleware and validators to prevent SQL injection and XSS attacks.
2. **Rate Limiting**: Multiple throttling classes for different endpoints to prevent abuse.
3. **Secure Headers**: Middleware for adding security headers to all responses.
4. **JWT Authentication**: Secure token-based authentication for both Django and NextJS.
5. **Index Strategies**: Recommended indexes for high write throughput on transaction tables.

### Bonus Features
1. **Referral System**: Unique 16-character referral codes for users to invite others.
2. **Streak Bonuses**: Increasing daily login bonuses for consistent usage.
3. **Content Access Control**: Check if users can access content based on subscription or credit balance.

### Database Considerations
- Added indexes for high write throughput scenarios
- Used transaction atomic to ensure data consistency
- Implemented connection pooling for better performance

## Next Steps for Frontend Implementation

For your NextJS frontend, you'll need to:

1. Create user profile page showing credits and subscription details
2. Implement content access controls based on the `/api/check-access/` endpoint
3. Design a credit transaction history view
4. Add referral code sharing functionality
5. Create subscription management UI

The backend is designed to work with your existing authentication system using JWT tokens, with secure handling of user data and proper role-based access controls.

Let me know if you'd like me to explain any specific part of the implementation in more detail!